In [1]:
import os
import re
import glob

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')


from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from num2words import num2words

#### PATH WHERE ALL THE FILES ARE THERE

In [2]:
path='C:/Users/anish_yagt7hg/Desktop/internship/Untitled Folder/1974_3493_bundle_archive'

In [4]:
filenames  =glob.glob(path +'/*.csv')

filenames

['C:/Users/anish_yagt7hg/Desktop/internship/Untitled Folder/1974_3493_bundle_archive\\articles1.csv',
 'C:/Users/anish_yagt7hg/Desktop/internship/Untitled Folder/1974_3493_bundle_archive\\articles2.csv',
 'C:/Users/anish_yagt7hg/Desktop/internship/Untitled Folder/1974_3493_bundle_archive\\articles3.csv']

#### CREATING A LIST OF DATAFRAMES

In [3]:
def list_of_dataframes(filenames):
    dfs=[]
    for file in range(len(filenames)):
        dfs.append( pd.read_csv(filenames[file],usecols=['title','content','author']))
    return dfs

In [5]:
articles_df=   pd.concat(list_of_dataframes(filenames))

In [6]:
articles_df.shape

(142570, 3)

In [7]:
articles_df.head()

title  \
0  House Republicans Fret About Winning Their Hea...   
1  Rift Between Officers and Residents as Killing...   
2  Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...   
3  Among Deaths in 2016, a Heavy Toll in Pop Musi...   
4  Kim Jong-un Says North Korea Is Preparing to T...   

                          author  \
0                     Carl Hulse   
1  Benjamin Mueller and Al Baker   
2                   Margalit Fox   
3               William McDonald   
4                  Choe Sang-Hun   

                                             content  
0  WASHINGTON  —   Congressional Republicans have...  
1  After the bullet shells get counted, the blood...  
2  When Walt Disney’s “Bambi” opened in 1942, cri...  
3  Death may be the great equalizer, but it isn’t...  
4  SEOUL, South Korea  —   North Korea’s leader, ...

#### CHECKING NULLS IN THE  AUTHOR COLUMN IF ANY

In [12]:
articles_df[articles_df['author'].isnull()]

title author  \
65                        My Canada - The New York Times    NaN   
66     How We Put Together Our 52 Places to Go List -...    NaN   
200    The Best and Worst of the Golden Globes - The ...    NaN   
260    President Obama’s Farewell Address: Full Video...    NaN   
367    Transcript: President Obama on What Books Mean...    NaN   
...                                                  ...    ...   
40576  Date Lab: Can two avid cooks make something to...    NaN   
41033  Date Lab: She’s a high-energy Coast Guard pilo...    NaN   
41380  Venezuela stops Post journalist, other corresp...    NaN   
41554  Pakistan deports National Geographic’s iconic ...    NaN   
41758  Cory Booker, Zephyr Teachout and more on the D...    NaN   

                                                 content  
65     Canada, our No. 1 pick for this year’s 52 Plac...  
66     For the 12th straight year, the Travel section...  
200    Here’s a look at the most memorable moments fr...  
260    President Obama delivered his farewell address...  
367    Michiko Kakutani, the chief book critic for Th...  
...                                                  ...  
40576        Interviews by Timothy Smith John Heinema...  
41033        Interviews by Amanda McGrath John Alexan...  
41380   Venezuelan authorities stopped a Washington P...  
41554    KABUL  —   Afghanistan’s president on Wednes...  
41758          In the wake of Republican victories on...  

[15876 rows x 3 columns]

In [13]:
articles_df=articles_df[~articles_df['author'].isnull()]   # SUBSETTING THE DATAFRAMES WHICH ARE NOT NULL

In [14]:
articles_df.head()

title  \
0  House Republicans Fret About Winning Their Hea...   
1  Rift Between Officers and Residents as Killing...   
2  Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...   
3  Among Deaths in 2016, a Heavy Toll in Pop Musi...   
4  Kim Jong-un Says North Korea Is Preparing to T...   

                          author  \
0                     Carl Hulse   
1  Benjamin Mueller and Al Baker   
2                   Margalit Fox   
3               William McDonald   
4                  Choe Sang-Hun   

                                             content  
0  WASHINGTON  —   Congressional Republicans have...  
1  After the bullet shells get counted, the blood...  
2  When Walt Disney’s “Bambi” opened in 1942, cri...  
3  Death may be the great equalizer, but it isn’t...  
4  SEOUL, South Korea  —   North Korea’s leader, ...

#### CREATING A DICTIONARY WITH AUTHOR AND ARTICLE COUNTS 

In [15]:
authors_counts=dict()
authors_list =articles_df['author'].values.tolist()

In [16]:
for i in authors_list:
    if i in authors_counts:
        authors_counts[i]+=1
    else:
        authors_counts[i]=1

In [17]:
authors_counts

{'Carl Hulse': 21,
 'Benjamin Mueller and Al Baker': 4,
 'Margalit Fox': 5,
 'William McDonald': 1,
 'Choe Sang-Hun': 37,
 'Sewell Chan': 14,
 'Javier C. Hernández': 8,
 'Gina Kolata': 18,
 'Katherine Rosman': 2,
 'Andy Newman': 10,
 'Justin Gillis': 4,
 'John Schwartz': 5,
 'Maggie Haberman': 30,
 'Charles Duhigg': 4,
 'Stephanie Rosenbloom': 6,
 'Emma G. Fitzsimmons': 11,
 'Kevin Sack and Alan Blinder': 4,
 'Geeta Anand': 6,
 'The Associated Press': 29,
 'Brett Cole': 6,
 'Benjamin Hoffman': 16,
 'Patrick Healy': 21,
 'Marlise Simons': 2,
 'Guy Trebay and Jacob Bernstein': 1,
 'Jennifer Steinhauer': 34,
 'Charles McDermid': 45,
 'Sheri Fink and Helene Cooper': 1,
 'Tim Arango': 22,
 'Mark Landler': 45,
 'Michelle Higgins': 7,
 'Isabel Kershner': 26,
 'Alissa J. Rubin': 14,
 'Ian Austen': 11,
 'Simon Romero': 18,
 'Kenneth Chang': 18,
 'Corey Kilgannon': 9,
 'Jennifer Schuessler': 15,
 'John Otis': 38,
 'Eric Lipton and Matt Flegenheimer': 1,
 'Sean Alfano': 6,
 'Jennifer Steinhauer a

#### FILTERING THE AUTORS WITH   ARTICLES  WITH CERTAIN NUMBER FOR SPEEDING UP THE PROCESS

In [18]:
filtered_author_names =[]
for k,v in authors_counts.items():
    if v>=100:
        filtered_author_names.append(k)

In [19]:
len(filtered_author_names)

223

In [20]:
articles_df=articles_df[articles_df['author'].isin(filtered_author_names)]

In [21]:
articles_df.head()

title        author  \
79    2 Credit-Reporting Agencies Must Pay $23 Milli...       Reuters   
3646  Harley-Davidson to Pay $12 Million Fine in Pol...       Reuters   
4630  Regulator Tightens Control Over Wells Fargo - ...       Reuters   
5469  Pope Francis Says Church Should Apologize to G...       Reuters   
7803  CNN’s Zeleny: ’Hard to Imagine’ Obama Would Ha...  Ian Hanchett   

                                                content  
79    The nation’s consumer watchdog agency on Tuesd...  
3646  WASHINGTON  —     has agreed to pay a $12 mill...  
4630  A leading bank regulator on Friday reversed co...  
5469  Pope Francis said on Sunday that Christians an...  
7803  On Tuesday’s broadcast of CNN’s “Situation Roo...

In [22]:
articles_df=articles_df.reset_index().drop('index',1)         

In [23]:
articles_df.shape

(50331, 3)

#### PREPROCESSING

In [24]:
def lower_data(string):
    return string.lower()

In [25]:
def remove_apostrophe(string):
    text=re.sub(r'[\']','',string)
    return text.lower()

In [26]:
def remove_punctuation(string):
    text=re.sub(r'[^\w\s]',' ',string)
    return text

In [27]:
def remove_stopwords(words):
    stop= stopwords.words('english')
    words=[word for word in words if word not in stop and len(word)>2]
    return words

In [28]:
def tokenize_words(string):
    string=string.strip()
    words=word_tokenize(string)
    return words

In [29]:
def stemming(words):
    stemmer =PorterStemmer()
    words=[stemmer.stem(word) for word in words]
    return words

In [30]:
def num_to_words(words):
    words=[num2words(int(word),to='ordinal')  if word.isdigit() else word for word in words]
    return words

In [31]:
def words_to_string(words):
    string =' '.join(word for word in words)
    return string


In [32]:
def preprocess(string):
    string = lower_data(string)
    string = remove_apostrophe(string)
    string = remove_punctuation(string)
    words = tokenize_words(string)
    words = remove_stopwords(words)  
    words = stemming(words)
    final_string = words_to_string(words)
    
    return final_string
    

#### APPLYING PREPROCESS FUNCTION TO ARTICLES

In [33]:
%time articles_df['content'] = articles_df['content'].apply(preprocess)

Wall time: 11min 34s


In [32]:
from sklearn.model_selection import train_test_split

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
vectorizer= TfidfVectorizer(max_features=20000,max_df=0.2,min_df=6,use_idf=True,ngram_range=(1,2))

In [35]:
X =articles_df['content']
Y=articles_df['author']

In [36]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.20,random_state=6025)

In [37]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(40264,)
(10067,)
(40264,)
(10067,)


In [38]:

X_train_tf = vectorizer.fit_transform(X_train).toarray()
X_test_tf = vectorizer.transform(X_test).toarray()

print(X_train_tf.shape)
print(X_test_tf.shape)

(40264, 20000)
(10067, 20000)


In [ ]:
from sklearn.decomposition import TruncatedSVD 
svd = TruncatedSVD(n_components=5000,n_iter=10,random_state=892)
SVD= svd.fit(X_train_tf)
%time X_train_tf = SVD.transform(X_train_tf)
print(svd.explained_variance_ratio_.sum())


In [39]:
X_test_tf = SVD.transform(X_test_tf)

In [40]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report

NBclassifier = GaussianNB()
NBclassifier.fit(X_train_tf, Y_train)

## Predictions
train_nb_preds = NBclassifier.predict(X_train_tf)
test_nb_preds = NBclassifier.predict(X_test_tf)

print("Train Accuracy", lassifier.score(X_train_tf,Y_train))
print(confusion_matrix(Y_train,train_nb_preds))
#print(classification_report(y_train,train_nb_preds))

print("Test Accuracy",NBclassifier.score(X_test_tf,Y_test))
print(confusion_matrix(Y_test,test_nb_preds))
#print(classification_report(y_test,test_nb_preds))

Train Accuracy 0.6718707501495947
[[444   0   0 ...   1   0   0]
 [  0 177   0 ...   0   0   0]
 [  0   0 132 ...   0   0   0]
 ...
 [  0   0   0 ... 270   0   0]
 [  0   0   0 ...   0 105   0]
 [  1   0   0 ...   0   0 258]]
Test Accuracy 0.4314621409921671
[[141   0   0 ...   0   0   0]
 [  0  75   0 ...   0   0   0]
 [  0   0  19 ...   0   0   0]
 ...
 [  0   0   0 ...  67   0   0]
 [  0   0   0 ...   0   8   0]
 [  1   0   0 ...   0   0  79]]


In [43]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(multi_class='ovr',solver='sag')
logit.fit(X_train_tf,Y_train)

## Predictions
train_logit_preds = logit.predict(X_train_tf)
test_logit_preds = logit.predict(X_test_tf)

print("Train Accuracy",logit.score(X_train_tf,Y_train))
print(confusion_matrix(Y_train,train_logit_preds))
#print(classification_report(y_train,train_logit_preds))

print("Test Accuracy",logit.score(X_test_tf,Y_test))
print(confusion_matrix(Y_test,test_logit_preds))
#print(classification_report(y_test,test_logit_preds))

Train Accuracy 0.7753903062612196
[[540   0   0 ...   0   0   0]
 [  0 208   0 ...   0   0   0]
 [  0   0 170 ...   0   0   0]
 ...
 [  0   0   0 ... 353   0   0]
 [  0   0   0 ...   0  56   0]
 [  1   0   0 ...   0   0 403]]
Test Accuracy 0.6733028720626631
[[178   0   0 ...   0   0   0]
 [  0  84   0 ...   0   0   0]
 [  0   0  35 ...   0   0   0]
 ...
 [  0   0   0 ... 103   0   0]
 [  1   0   0 ...   0   6   0]
 [  0   0   0 ...   0   0 133]]


In [45]:
from sklearn.ensemble import RandomForestClassifier

In [48]:
from sklearn.metrics import accuracy_score

In [46]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import recall_score, precision_score, f1_score, make_scorer,
score_metric = make_scorer(f1_score, pos_label='POS')
## n_jobs = -1 uses all cores of processor
## max_features is the maximum number of attributes to select for each tree
rfc_grid = RandomForestClassifier(n_jobs=-1, max_features='sqrt', class_weight='balanced')
 
# Use a grid over parameters of interest
## n_estimators is the number of trees in the forest
## max_depth is how deep each tree can be
## min_sample_leaf is the minimum samples required in each leaf node for the root node to split
## "A node will only be split if in each of it's leaf nodes there should be min_sample_leaf"

param_grid = {"n_estimators" : [10, 25, 50, 75, 100],
           "max_depth" : [np.arange(4,16,2)],
           "min_samples_leaf" : [5, 10, 15, 20],
             "criterion":['entropy','gini']}
 
rfc_cv_grid = RandomizedSearchCV(estimator = rfc_grid, param_distributions = param_grid, cv = 6, n_iter=25, n_jobs=-1)
rfc_cv_grid.fit(X_train_tf, Y_train)
rfc_cv_grid.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=16, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=10,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=75, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [ ]:
rfc_train_pred = rfc_cv_grid.best_estimator_.predict(X_train_tf)
rfc_test_pred = rfc_cv_grid.best_estimator_.predict(X_test_tf)

print("Train")
print(accuracy_score(Y_train,rfc_train_pred))
print(confusion_matrix(Y_train,rfc_train_pred))



print("Test")
print(accuracy_score(Y_test,rfc_test_pred))
print(confusion_matrix(Y_test,rfc_test_pred))